In [1]:
import pandas as pd
import numpy as np
import datetime

## Limpieza de datos del dataset Drones (PRA2 - Visualización de Datos)

In [2]:
df = pd.read_csv('../09 - Creación de la visualización y entrega proyecto (PRA2)/ASNdronesdb.csv', encoding="latin-1", sep=",")

# Usamos encoding latin-1 porque el que viene por defecto "utf-8" da error a la hora de leer el dataset

In [3]:
df.shape

(16750, 15)

In [4]:
# Mostramos las 5 primeras filas del dataset
df.head()

,Datetime,City,State,Narrative,Altitude,Separation,Aircraft,Flight_nr,Operator,id,Evasive_man,Source,Country,AircraftCategory,ReportId
0,1997-08-03,Schopfheim,NaN,Collided in mid-air with a remote controlled a...,650,0,Grob G109B,NaN,NaN,2200,0,https://aviation-safety.net/wikibase/wiki.php?...,Germany,G,3X306-1/2/97
1,2004-06-24 15:21:00,"Montréal/St-Hubert, QC (CYHU)",NaN,"C-FTSJ, a Cargair Piper Aztec, reported that a...",NaN,NaN,Piper PA-23-250 Aztec,C-FTSJ,Cargair,3461,0,TC,Canada,G,2004Q0848
2,2004-07-30 22:35:00,"Boundary Bay, BC (CZBB)",NaN,The pilot of C-FTUK a privately registered Rob...,NaN,NaN,Robinson R44,C-FTUK,Private,3460,0,TC,Canada,H,2004P1375
3,2006-02-08 23:09:00,CAMPBELL RIVER BC (CYBL),NaN,The pilot of C-GHLZ a Robinson R22 owned by E ...,1000,NaN,Robinson R22,NaN,E & B Helicopters,3463,0,TC,Canada,H,2006P0176
4,2006-06-17 16:22:00,ROBERVAL QC (CYRJ),NaN,"At 1622Z, when CGWGF was approximately 5 NM on...",NaN,NaN,Cessna TR182,C-GWGF,NaN,3464,0,TC,Canada,G,2006Q1193


In [5]:
df.City.value_counts()

NEW YORK                            587
LOS ANGELES                         374
CHICAGO                             305
HOUSTON                             300
New York                            275
                                   ... 
Peoria                                1
5305N 00002W 2nm E Coningsby ATZ      1
Sydney Aerodrome, 040° T 11Km         1
Southbridge                           1
Lions Gate bridge, BC                 1
Name: City, Length: 3415, dtype: int64

In [6]:
# Eliminamos los registros que no tienen ciudad (en total son 27)
df_without_city = df.dropna(subset=['City'])
df_without_city.shape

(16723, 15)

In [7]:
df.Aircraft.value_counts()

Cessna 172                1434
Piper PA-28                762
Airbus A320                747
Boeing 737                 670
Boeing 737-800             543
                          ... 
ATR 42-500                   1
Sikorsky S-76D               1
Thorp T-18                   1
Luscombe 8                   1
Bombardier BD-700-1A10       1
Name: Aircraft, Length: 1105, dtype: int64

In [8]:
# Eliminamos los registros que no tienen valor en aeronave (en total son 302)
df_without_acft = df_without_city.dropna(subset=['Aircraft'])
df_without_acft.shape

(16421, 15)

In [9]:
# Eliminamos la fila con valor NaN en la fuente (solo hay 1)
df_without_source = df_without_acft.dropna(subset=['Source'])
df_without_source.shape

(16420, 15)

In [10]:
# Eliminamos la fila con valor NaN en en el país (solo hay 1)
df_without_country = df_without_source.dropna(subset=['Country'])
df_without_country.shape

(16419, 15)

In [11]:
# Del campo AircraftCategory, eliminamos todos aquellos registros que no contengan los valores categóricos A, G, H, B o M:
df_acft_category = df_without_country[df_without_country["AircraftCategory"].str.contains("A|G|H|B|M") == True]
df_acft_category.shape

(16292, 15)

In [12]:
List_Of_Categories_In_Column = list(df_acft_category['Source'].value_counts().index)

In [13]:
List_Of_Categories_In_Column

['FAA',
 'TC',
 'ATSB',
 'UKAB',
 'ANSV',
 'ASRS',
 'FAA;ASRS',
 'https://aet.gouvernement.lu/dam-assets/l-administration/aviation-civile/rapport-lx-lgg-uav-20180508-post-consultation-v2.pdf',
 'STSB',
 'TSB',
 'https://uzpln.cz/pdf/20221020094447.pdf',
 'FAVT',
 'Dutch Safety Board',
 'LiveATC',
 'UZPLN',
 'http://rr.sapo.pt/noticia/87173/incidente_com_drone_sexto_caso_so_em_junho',
 'NBAAI',
 'https://newyork.cbslocal.com/2019/01/22/drones-newark-airport-teterboro/',
 'https://aviation-safety.net/wikibase/wiki.php?id=58711',
 'https://turvallisuustutkinta.fi',
 'https://www.geo.tv/latest/225585-sukkur-bound-flight-has-a-close-shave-with-a-drone-over-karachi-airport',
 'https://www.airlineratings.com/news/virgin-plane-reports-close-shave-drone/',
 'https://cnycentral.com/news/nation-world/sheriff-drone-flew-over-helicopter-dumping-water-on-oregon-wildfire',
 'https://www.stuff.co.nz/national/109698433/call-for-public-assistance-after-police-eagle-helicopters-nearmiss-with-drone',
 'ht

Así pues, nos quedaremos únicamente con aquellas filas cuya fuente sea un organismo oficial (normalmente, aquellos valores compuestos por siglas), y se descartarán todos aquellos valores cuya fuente sea un enlace a una página web o un enlace de Twitter. De esta manera, mediante una Expresión Regular, se pueden eliminar todos aquellos registros cuya fuente contenga un `http` o códigos extraños como `BFU15-0913-6X`:

In [14]:
# Se identifica el string parcial
descarte = ['http', 'BFU15-0913-6X']
  
# Eliminamos aquellas filas que contienen el string parcial "http":
df_http = df_acft_category[~df_acft_category.Source.str.contains('|'.join(descarte))]
df_http.shape

(16214, 15)

In [15]:
df_http.Source.value_counts()

FAA                   13791
TC                      844
ATSB                    656
UKAB                    510
ANSV                    223
ASRS                    107
FAA;ASRS                 35
STSB                     11
TSB                       8
UZPLN                     4
Dutch Safety Board        4
FAVT                      4
LiveATC                   4
NBAAI                     3
NTSB;FAA                  1
ANP                       1
SUST                      1
AIAI                      1
S.A. CAA                  1
BEA                       1
OVV                       1
AIBN                      1
BFU                       1
NTSB                      1
Name: Source, dtype: int64

Ya está el dataset casi limpio. Están todos los registros eliminados con valores nulos o no válidos; ahora solo falta eliminar las columnas que no aportan información relevante o que tienen demasiados valores faltantes (NA) respecto al número total:

In [16]:
df_clean = df_http.drop(columns=['Separation', 'Flight_nr', 'Operator', 'ReportId'])

In [17]:
df_clean.head()

,Datetime,City,State,Narrative,Altitude,Aircraft,id,Evasive_man,Source,Country,AircraftCategory
1,2004-06-24 15:21:00,"Montréal/St-Hubert, QC (CYHU)",NaN,"C-FTSJ, a Cargair Piper Aztec, reported that a...",NaN,Piper PA-23-250 Aztec,3461,0,TC,Canada,G
2,2004-07-30 22:35:00,"Boundary Bay, BC (CZBB)",NaN,The pilot of C-FTUK a privately registered Rob...,NaN,Robinson R44,3460,0,TC,Canada,H
3,2006-02-08 23:09:00,CAMPBELL RIVER BC (CYBL),NaN,The pilot of C-GHLZ a Robinson R22 owned by E ...,1000,Robinson R22,3463,0,TC,Canada,H
4,2006-06-17 16:22:00,ROBERVAL QC (CYRJ),NaN,"At 1622Z, when CGWGF was approximately 5 NM on...",NaN,Cessna TR182,3464,0,TC,Canada,G
5,2006-07-24 22:50:00,SEPT-ÎLES QC (CYZV),NaN,Model airplane activity was reported by a Cess...,NaN,Cessna 310,3533,0,TC,Chile,G


El dataset ya está limpio. Solo falta un último detalle: la columna 'Datetime', conviene separarla en dos: una para mostrar la fecha y otra columna para mostrar la hora. Antes que nada, se van a eliminar aquellos registros que tienen `00` en el día; por ejemplo, hay registros cuya fecha es `2006-08-00` y no son válidos. Estos serán eliminados:

In [18]:
df1 = df_clean[df_clean["Datetime"].str.contains("-00|  ") == False]  # OJO: False, para eliminar los registros
df1.shape

(16105, 11)

Se han eliminado **109 registros** que contenían fechas con un `00` en el día de la fecha. Ahora seguimos cribando: lo siguiente será dividir la columna `Datetime` en dos columnas nuevas diferentes: una para la fecha (`Date`) y otra para la hora (`Time`):

In [19]:
# Dividimos la columna Datetime en dos variables separadas
df1['Date'] = pd.to_datetime(df1['Datetime']).dt.date
df1['Time'] = pd.to_datetime(df1['Datetime']).dt.time

# Una vez dividida, eliminamos la variable Datetime ya que es redundante:
df2 = df1.drop(columns=['Datetime'])

C:\Users\romej\AppData\Local\Temp\ipykernel_18248\2301633801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date'] = pd.to_datetime(df1['Datetime']).dt.date
C:\Users\romej\AppData\Local\Temp\ipykernel_18248\2301633801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Time'] = pd.to_datetime(df1['Datetime']).dt.time


Por último, pasamos los valores de la fecha `Date` de formato `yyyy-mm-dd` a `dd-mm-yyyy`: 

In [20]:
# df2["Date"] = pd.to_datetime(df2['Date'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')

In [21]:
df2.Time.value_counts()

00:00:00    12535
17:00:00       22
16:00:00       20
14:00:00       18
18:30:00       18
            ...  
05:20:00        1
05:42:00        1
03:52:00        1
17:16:00        1
02:54:00        1
Name: Time, Length: 1010, dtype: int64

Como hay 12535 registros con valores nulos en la columna "tiempo" (`Time`), se eliminará este atributo y se dejará únicamente el de la fecha:

In [22]:
drones = df1[['Date', 'City', 'State', 'Country', 'Narrative', 'Source', 
              'Altitude', 'Aircraft', 'AircraftCategory', 'Evasive_man']]

In [23]:
drones

,Date,City,State,Country,Narrative,Source,Altitude,Aircraft,AircraftCategory,Evasive_man
1,2004-06-24,"Montréal/St-Hubert, QC (CYHU)",NaN,Canada,"C-FTSJ, a Cargair Piper Aztec, reported that a...",TC,NaN,Piper PA-23-250 Aztec,G,0
2,2004-07-30,"Boundary Bay, BC (CZBB)",NaN,Canada,The pilot of C-FTUK a privately registered Rob...,TC,NaN,Robinson R44,H,0
3,2006-02-08,CAMPBELL RIVER BC (CYBL),NaN,Canada,The pilot of C-GHLZ a Robinson R22 owned by E ...,TC,1000,Robinson R22,H,0
4,2006-06-17,ROBERVAL QC (CYRJ),NaN,Canada,"At 1622Z, when CGWGF was approximately 5 NM on...",TC,NaN,Cessna TR182,G,0
5,2006-07-24,SEPT-ÎLES QC (CYZV),NaN,Chile,Model airplane activity was reported by a Cess...,TC,NaN,Cessna 310,G,0
...,...,...,...,...,...,...,...,...,...,...
16742,2022-09-27,CHARLOTTE,NORTH CAROLINA,USA,"PRELIM INFO FROM FAA OPS: CHARLOTTE, NC/UAS SI...",FAA,9000,CRJ-900,A,0
16744,2022-09-29,"Lions Gate bridge, BC",NaN,Canada,A Gulf Island Seaplanes de Havilland DHC-2 MK....,TC,800,DHC-2,A,0
16746,2022-10-05,"Toronto/Lester B. Pearson, ON (CYYZ)",NaN,Canada,A WestJet Boeing 737-7CT (C-GWSN/WJA660) from ...,TC,NaN,Boeing 737-7CT,A,0
16747,2022-10-18,"Calgary, AB (CYYC)",NaN,Canada,A WestJet Boeing 737-700 (WJA267) from Winnipe...,TC,NaN,Boeing 737-7CT,A,0


---

Vamos a centrarnos ahora en la única variable numérica que ha quedado: `Altitude` (expresada en pies `ft.`). Primero vamos a unificar criterios, y dejar solo un número, puesto que hay valores expresados como `1000-1200`. Vamos a dejar solo el primer valor y eliminar todo lo que venga después del guion:

In [24]:
# Eliminamos todos los números después del guion para unificar criterios
drones['Altitude'] = drones['Altitude'].str.split('-').str[0]

C:\Users\romej\AppData\Local\Temp\ipykernel_18248\3028588228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drones['Altitude'] = drones['Altitude'].str.split('-').str[0]


In [25]:
drones.Altitude.value_counts()

3000     634
2000     618
4000     476
1000     436
1500     416
        ... 
28200      1
9600       1
330        1
26         1
16800      1
Name: Altitude, Length: 229, dtype: int64

De todos los valores, hay tres que no son numéricos: `WHILE AT13`, `FL180` y `BOUND AT15`. Averiguamos el índice de esos valores, es decir, qué posición o fila ocupan en el dataset, y los borramos del conjunto.
NOTA: lo más fácil sería eliminar todos aquellos valores no numéricos mediante RegEx, pero también eliminan los NaN values.

In [26]:
a = drones[((drones.Altitude == 'WHILE AT13'))].index
b = drones[((drones.Altitude == 'FL180'))].index
c = drones[((drones.Altitude == 'BOUND AT15'))].index

In [27]:
drones = drones.drop(a)
drones = drones.drop(b)
drones = drones.drop(c)

In [28]:
# Una vez limpiado, convertimos la columna Altitud en numérica en lugar de string:
drones["Altitude"] = pd.to_numeric(drones["Altitude"])

In [29]:
drones

,Date,City,State,Country,Narrative,Source,Altitude,Aircraft,AircraftCategory,Evasive_man
1,2004-06-24,"Montréal/St-Hubert, QC (CYHU)",NaN,Canada,"C-FTSJ, a Cargair Piper Aztec, reported that a...",TC,NaN,Piper PA-23-250 Aztec,G,0
2,2004-07-30,"Boundary Bay, BC (CZBB)",NaN,Canada,The pilot of C-FTUK a privately registered Rob...,TC,NaN,Robinson R44,H,0
3,2006-02-08,CAMPBELL RIVER BC (CYBL),NaN,Canada,The pilot of C-GHLZ a Robinson R22 owned by E ...,TC,1000.0,Robinson R22,H,0
4,2006-06-17,ROBERVAL QC (CYRJ),NaN,Canada,"At 1622Z, when CGWGF was approximately 5 NM on...",TC,NaN,Cessna TR182,G,0
5,2006-07-24,SEPT-ÎLES QC (CYZV),NaN,Chile,Model airplane activity was reported by a Cess...,TC,NaN,Cessna 310,G,0
...,...,...,...,...,...,...,...,...,...,...
16742,2022-09-27,CHARLOTTE,NORTH CAROLINA,USA,"PRELIM INFO FROM FAA OPS: CHARLOTTE, NC/UAS SI...",FAA,9000.0,CRJ-900,A,0
16744,2022-09-29,"Lions Gate bridge, BC",NaN,Canada,A Gulf Island Seaplanes de Havilland DHC-2 MK....,TC,800.0,DHC-2,A,0
16746,2022-10-05,"Toronto/Lester B. Pearson, ON (CYYZ)",NaN,Canada,A WestJet Boeing 737-7CT (C-GWSN/WJA660) from ...,TC,NaN,Boeing 737-7CT,A,0
16747,2022-10-18,"Calgary, AB (CYYC)",NaN,Canada,A WestJet Boeing 737-700 (WJA267) from Winnipe...,TC,NaN,Boeing 737-7CT,A,0


Un último detalle: en la columna `City` y `State` hay muchos valores repetidos porque a veces el estado está escrito en mayúsculas y a veces en minúsculas, de tal forma que podemos encontrar `WASHINGTON` y `Washington`. Para unificar criterios, se van a convertir todos los nombres de Estados en mayúsculas, así no habrá duplicidades:

In [30]:
drones["State"] = drones["State"].str.upper()

In [31]:
drones["City"] = drones["City"].str.upper()

In [32]:
# Eliminamos duplicidades al eliminar todas aquellas ciudades que se llamaban igual
# pero que añadían las coordenadas o algún elemento innecesario después de una coma
drones['City'] = drones['City'].str.split(',').str[0]

In [35]:
# Eliminamos duplicidades al eliminar todas aquellas ciudades que se llamaban igual
# pero que añadían las coordenadas o algún elemento innecesario después de una coma
drones['City'] = drones['City'].str.split('AERODROME').str[0]

In [36]:
drones

,Date,City,State,Country,Narrative,Source,Altitude,Aircraft,AircraftCategory,Evasive_man
1,2004-06-24,MONTRÉAL/ST-HUBERT,NaN,Canada,"C-FTSJ, a Cargair Piper Aztec, reported that a...",TC,NaN,Piper PA-23-250 Aztec,G,0
2,2004-07-30,BOUNDARY BAY,NaN,Canada,The pilot of C-FTUK a privately registered Rob...,TC,NaN,Robinson R44,H,0
3,2006-02-08,CAMPBELL RIVER BC (CYBL),NaN,Canada,The pilot of C-GHLZ a Robinson R22 owned by E ...,TC,1000.0,Robinson R22,H,0
4,2006-06-17,ROBERVAL QC (CYRJ),NaN,Canada,"At 1622Z, when CGWGF was approximately 5 NM on...",TC,NaN,Cessna TR182,G,0
5,2006-07-24,SEPT-ÎLES QC (CYZV),NaN,Chile,Model airplane activity was reported by a Cess...,TC,NaN,Cessna 310,G,0
...,...,...,...,...,...,...,...,...,...,...
16742,2022-09-27,CHARLOTTE,NORTH CAROLINA,USA,"PRELIM INFO FROM FAA OPS: CHARLOTTE, NC/UAS SI...",FAA,9000.0,CRJ-900,A,0
16744,2022-09-29,LIONS GATE BRIDGE,NaN,Canada,A Gulf Island Seaplanes de Havilland DHC-2 MK....,TC,800.0,DHC-2,A,0
16746,2022-10-05,TORONTO/LESTER B. PEARSON,NaN,Canada,A WestJet Boeing 737-7CT (C-GWSN/WJA660) from ...,TC,NaN,Boeing 737-7CT,A,0
16747,2022-10-18,CALGARY,NaN,Canada,A WestJet Boeing 737-700 (WJA267) from Winnipe...,TC,NaN,Boeing 737-7CT,A,0


---

Ya está hecha la limpieza de datos. Ahora se puede exportar el nuevo dataset limpio y filtrado:

In [37]:
drones.to_excel(r'../09 - Creación de la visualización y entrega proyecto (PRA2)/DRONES.xlsx', sheet_name='DRONES', index=False)